# **Chapter 03-d. Other Convolutional Neural Network Models**

* 這一章, 我們會談到除了 AlexNet 與 MobileNet 以外的其他 Convolutional Network Models
* 包含 **AlexNet, VGG19, Inception V3, SqueezeNet, ResNet, DenseNet, Xception, NASNet, MobileNet, EfficientNet, 及 Vision Transformer (ViT)**

### **為什麼 Models 要有多層結構？**
* 單層是一個線性函數, 無論它有多少參數, 都無法趨近複雜的非線性函數
* 將參數添加到單一層會增加神經網路的記憶, 允許他學習到更複雜的東西, 但是他傾向於記住範例而學習他們, 而沒有做到良好的泛化; 堆疊多層, 會迫使模型將輸入分解為階層式結構, 例如: 初始層可以識別皮毛與鬍鬚, 隨後的層將它們組合而識別貓頭, 然後識別整隻貓
* 如果某一特徵在圖中的佔比很大, 單一 filter 有可能因為本身太小而無法捕獲他; 多層結構會從小的特徵向上組合, 可以改善這個問題

### **過濾器因式分解 (Filter Factorization)**
* 在實務上, `兩個 3×3 的 filter 層, 會比單一層 5×5 的 filter 之效果還要好`
* 因為 兩個 3×3 的 filter 有 2×3×3=18 個 weights, 而 單一個 5×5 有 5×5=25 個 weights 要訓練, 所以 `兩個 3×3 filter 的訓練成本較低`
* 再加上兩層filter會有兩個activation function, 可以有更好的表達能力 `(activation function 是網路中唯一的非線性部分！)`

### **1×1 Filter?**
* `1×1 的 filter 適合用來調整資料頻道數！`例如: RGB 影像上的三個色彩 channels 之間的組合可能會產生有用資訊, 但一般的 filter 只做用在單一頻道上, 這時我們可以利用 有三個參數的 1×1 filter 去算出這三個 channel 分別乘上參數的內積(點積)值, 並且重複多次, 就可以改變及控制 input 的 channel 數量了！
* 並且 1×1 filter 的 weights 數量跟其他 filter 比起來少非常非常多

### **Global Average Pooling (全域平均池化)**
* 假設任務是 N 種類別的 classification, 那麼 global average pooling 的概念上就是讓 `feature map 最後結束時有 N 個 channels, 分別對應到分類的 N 種類別`
* 通常 Global average pooling 後面會直接接上 softmax activation function, 或是有 N 個節點的 fully-connected layer w/ softmax activation function
* 而 global average pooling 就是`對整個 channel 的值取平均`, 所以本身是沒有 trainable weight 的, 相當便宜！
* 不過, 因為是對整個 channel 平均, 所以`位置資訊會消失` -> 如果今天是想找圖中有沒有特定資訊, 而不太在意他到底在哪裡的話, 就可以使用它


### **Depth-Seperable Convolution(深度可分離卷積)**
* 假設 input 有8個channels, 想要轉成 16 個channels的output
* 傳統方法: 使用 16 個 3×3×8 的 filters, 權重數量 = (3×3×8)×16 = 1152
* `使用可分離的卷積層`: (1) 把 3×3×8 的 filter `看作是 8個3×3 filters`, 分別去做用在對應到的那一層後疊在一起, 會得到8層的結構, (2) 再利用 `16 個 1×1×8 的 filters`, 就可以得到 16 channels 的 output, 權重數量 = (3×3)×8 + (1×1×8)×16 = 200, 明顯更少！
* `Depth multiplier (深度乘數)`: 上述的(1)步驟可以重複不止一次, 得到的會是更多倍的頻道數, 不過通常都不會重複做

---
## **AlexNet**, <small>2012</small> (Ch03-c 介紹過)
* Alexnet 是 classification 領域最早的成功案例之一, 他的結構如下圖：
* 每個卷積層的 activation function 都是選用 ReLU
* 經過 pool5 之後, 會先 Flatten(), 再經過三個 fully-connected network, 最後經由 softmax 做出預測！
 
![alt text](Images/AlexNet.jpg "AlexNet_structure")

---
## **VGG19**, <small>2014</small>
* 共 19 層, 除了最後一層使用 softmax 外, 其他層的 activation function 都是 ReLU
* 與 AlexNet 相比, 它使用了更多的卷積層 (16 vs 5), 全部都是 3×3 的 filter
* 不過, VGG19 仍然使用與 AlexNet 一樣的 prediction head (最後三層都是 fully-connected layer), 這三層就有1.2億個 weights...

<!-- ![alt text](Images/VGG19.webp "VGG19_structure") -->
<img src="Images/VGG19.webp" width="750"/>

---
## **Inception V3**, <small>2015</small>
* 由非常多的模組(module)組成, 架構非常複雜, 主要的概念是`提供很多的卷積層與池化層之組合, 讓模型在訓練時自己去學習什麼樣的組合是最好的`
* 可以想像成, 模型在下圖中走什麼路徑會有最好的成果
* 因為模型更深, 所以可以做到比 AlexNet 與 VGG19 更好的成果, 但後來已經有非常多更新、更簡單的替代方法了

![alt text](Images/InceptionV3.png "AlexNet_structure")

---
## **SqueezeNet**, <small>2016</small>
* `Fire module`先使用 1×1 filter 對影像進行擠壓(squeeze), 使它的channel數減少, 再同時給 1×1 與 3×3 的 filter 處理, 最後進行串接
* SqueezeNet 即是 `fire module 與 max pooling layer 的交替排列`
* Maxpool 層是 2×2 的 max pooling, strides = 2; Conv 層都是用 ReLU 當作 activation function
* 最後一層 conv 會使用 1×1 的 filter 將 channel 數提高至 1000, 再對 1000 個頻道做 global average pooling 並接到 softmax activation function
* SqueezeNet 的優點是簡單且權重不多, 但缺點是第一層 conv 使用 7×7 的 filter, 並不是最佳實務的架構(靈感來自於AlexNet)

<!-- <img src="Images/SqueezeNet.png" alt="SqueezeNet_structure" width="600"/>
<img src="Images/SqueezeNet_2.png" alt="SqueezeNet_structure" width="600"/> -->

<!-- |![](Images/SqueezeNet.png)|![](Images/SqueezeNet_2.png)|
|:-:|:-:|
|Fire Module|SqueeezeNet structure| -->

<p float="left">
  <img src="Images/SqueezeNet.png" width="750" />
  <img src="Images/SqueezeNet_2.png" width="300" /> 
</p>

---
## **ResNet**, <small>2015</small>
* 改善極深度網路常見的問題 - 梯度消失（vanishing gradients）, 模型太深會導致模型難以收斂, 尤其`當網路層數很深時, 梯度在反向傳播(backpropagation)過程中會逐層變小甚至消失`
* ResNet 使用 `skip connection（殘差連接）`來解決上述情況, 繞過部分層級, 將輸入 x 直接加回輸出 C(x), 執行 element-by-element 加法：f(x) = C(x) + x
* `Residue (殘差)`: C(x) = f(x) - x, C(x) 為通過卷積層的輸出, x 為跳過連接, f(x) 為兩者之和; `C(x), 也就是 residue, 是期望輸出與輸入之間的差異`, 也是模型的學習方向(這樣的設計讓模型更容易學習接近 0 的變化, 提升收斂速度與效能)
* 因為 element-by-element 加法要在資料維度保持不變的情況才能執行, 所以 `residual block (殘差區塊)` 必須保留資料的 width, height & channels, 或是在 skip connection 路徑上也經過 conv layer 去調整輸出維度
* ResNet 架構可以堆疊許多 residual block, 常見的有 ResNet50(下圖) 和 ResNet101 等
* 所有 conv layer 都使用 ReLU activation function & bacth normalization
* Skip connection 優點: 在優化(backpropagation)時有助於梯度流過網路, 這讓 `ResNet 成為目前最具代表性的深層網路架構之一, 也常作為新模型設計的 baseline`

<img src="Images/ResNet50.png" width="300" />

---
## **DenseNet**, <small>2016</small>
* 與 ResNet 的 element-by-element 加法不同, DenseNet 的 skip connection 是將前幾層的輸出`串接`到後幾層上(`channel-wise concatenation`), 形成`dense block (密集區塊)`
* 所有 conv layer 一樣是使用 ReLU + batch normalization, 且`為了能夠串接, filter 皆使用 strides=1 不會改變 width & height`
* 下圖是一個 `dense block` 的架構, DesneNet 即是串接多個 dense block 的模型, dense block 之間使用 pooling layer (通常是 average pooling) 連接
* DenseNet 的 weights 數量其實比想像中少很多！DenseNet 的架構允許特徵被重複利用, 所以`每層 conv layer 需要的 filter 可以變得很少`, 是個很經濟的模型

<img src="Images/DenseNet_block.png" width="700" />

---
## **Xception**, <small>2016</small>
* 其實跟 ResNet 非常相似, 只是`將 ResNet使用的3×3與1×1傳統 conv layer組合, 替換成 3×3 depth-seperable conv layer` (Think: 3×3 depth-seperable conv layer本來就是3×3與1×1傳統 conv layer的組合了！)
* 因為是使用 depth-seperable conv layer, 所以需要訓練的權重數量更少！

<img src="Images/Xception.jpg" width="700" />

---
## **NASNet (Neural Architecture Search Net)**, <small>2017</small>
* 架構`自動搜尋（AutoML）`：NASNet 利用神經網路（稱為 Controller RNN）自動搜尋最佳網路架構，`不是人類來設計網路結構，而是讓神經網路自己學習設計更好的網路`。
* 重複使用「模組化 Cell」：NASNet 將搜尋限制在兩種可重複堆疊的模組（Normal Cell 會保持 feature map 的空間大小 & Reduction Cell 會降低 feature map 的空間大小(strides=2)），大幅降低搜尋成本並提升可遷移性。
* 使用 `Reinforcement Learning（強化學習）優化架構`：Controller 透過 RL 根據驗證集準確率回饋，不斷調整產生的 Cell 結構。
* 可遷移性高（Transferable）：在小資料集（如 CIFAR-10）上搜尋得到的 Cell，可直接應用在大型資料集（如 ImageNet）並取得佳績。
* 效能卓越但計算成本高：NASNet 模型在準確率上達到 SOTA，但搜尋過程需大量計算資源與時間（幾百個 GPU 天數）, 且權重多代表訓練資料的數量也需要更多
* 黑盒問題：雖然能找到好架構，但我們不容易解釋為什麼這樣設計是好的
* 後續影響深遠：啟發了後來更高效的 NAS 技術，如 DARTS、EfficientNet（結合 NAS + compound scaling）等。

---
## **MobileNet**, <small>2018</small>
* MobileNet 的兩大積木: **depthwise convolution** 和 **inverted residual bottleneck**
* `depthwise convolution (逐深度卷積)`: 即是 depth-seperable conv 的前半段, filter 分別作用在對應 channel 上的這部分
* `inverted residual bottleneck (反向殘差瓶頸)`: 與 ResNet 的 1×1 -> 3×3 -> 1×1 會讓頻道由多變少再變多相反, MobileNet 的 1×1 -> 3×3 -> 1×1 會讓頻道由少變多再變少(詳情見下圖), 這樣的設計會讓權重從 ResNet 的 1.1M 下降到 52k！
* 整體的模型架構是 inverted residual bottleneck 的堆疊, 不過細節方面有很多變化, 而`權重顯著下降的缺點就是準確度也略為下降, 且收斂速度較慢`, 但模型很輕量, 適用於行動端

<img src="Images/MobileNetV2.png" width="700"/>

---
## **EfficientNet**, <small>2019</small>
* 類似 NASNet, 讓神經網路去自動搜尋最佳的模型架構, 而 EfficientNet 將 inverted residual bottleneck 也加入了搜尋的基本積木裡
* EfficientNet 是一整個家族, 裡面有許多大大小小的變化型(層數不同/每一層頻道數不同/輸入影像解析度不同)
* 詳細介紹可以回去閱讀書中 p.119~124 的內容

---
## **ViT (Vision Transformer)**, <small>2020</small>
* 利用 transformer 的想法來處理影像, 對`像素塊(patch)`進行 `positional encoding`, 形成 (patch, position) 的表達法, 再經由 transformer 區塊來訓練
* 與其他卷積模型相比, ViT 需要的訓練資料量非常的多, 通常建議在大量資料上進行 pre-training, 再在較小的資料集上進行 fine-tuning

---
## **各模型之比較**

| Model            | Params | 特點               | 適用場景        |
| ---------------- | ------ | ---------------- | ----------- |
| **AlexNet**      | 多      | 第一個深度 CNN        | 教學、歷史       |
| **VGG19**        | 非常多    | 結構單純，遷移學習佳       | 特徵提取        |
| **Inception**    | 中      | 多尺度卷積            | 中大型任務       |
| **SqueezeNet**   | 少      | 超小模型             | 嵌入式         |
| **ResNet**       | 中高     | Skip connection  | 通用分類、檢測     |
| **DenseNet**     | 中      | Dense connection | 醫療、分類       |
| **Xception**     | 中      | 深度可分離卷積          | 高效推論        |
| **NASNet**       | 高      | 自動設計             | 高精度需求       |
| **MobileNet**    | 少      | 輕量、高效            | 行動端         |
| **EfficientNet** | 中      | 最佳效能/資源比例        | 實務部署        |
| **ViT**          | 高      | 全域特徵學習           | 大數據分類、預訓練任務 |


---
### **Ensemble (集成)**
* 組合多個模型的預測結果, 推薦要盡可能選擇架構差異性最大的模型來做組合(因為不同模型會有不一樣的弱點, 組合起來可以互補)
* 預測結果可以是他們預測類別的機率平均值, 或者最好的做法是 `logit 取平均值後再套上 softmax` 來計算個別機率！

---
## ⭐️ **推薦策略(經驗法則, 不是必然)**

考慮訓練方式時：
* 若資料集非常小 (`<1000 images per class`): 採用 `transfer learning`
* 若資料集為中等大小 (`1000~5000 images per class`): 採用 `fine-tuning`
* 若是大型資料集 (`>5000 images per class`): 考慮`從頭開始訓練`

考慮模型架構時：
* 想`設計自己的層`, 可以從 `SqueezeNet` 開始, 它是效能良好的最簡單模型
* 對於邊緣設備, 講求`空間小、低延遲、低功耗`時, 考慮` MobileNet V2`
* `無限制之下想要最好的模型`, 考慮 `EfficientNet`
* 保守一點, 想要使用`被驗證過的模型架構`, 考慮使用 `ResNet50` 或更大的變體
* 不擔心成本 & 延遲問題, 且模型的小幅改進可以得到很高回報的話, 考慮使用 `ensemble`